# Introduction

Provided the data downloaded from www.euets.info as zip file, we extract the data using the *ziploader* sub-module:

# Packages and options

In [1]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from pyeutl.ziploader import (get_installations, get_accounts, get_transactions,
                              get_compliance, get_account_holders) 

from pyeutl.ziploader.utils import (load_zipped_file, map_if_exists, get_mapper)
from pyeutl.ziploader.category_mappings import (map_activity_category, map_account_category,
                              map_nace_category)
%load_ext autoreload
%autoreload 2

In [2]:
fn_zip = "./eutl.zip"

# Get Data

Get installation data from zip file. The function merges information from lookup tables into the installation table:

In [3]:
df_inst = get_installations(fn_zip, )
df_inst.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17821 entries, 0 to 17820
Data columns (total 32 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   17821 non-null  object 
 1   name                 16205 non-null  object 
 2   registry_id          17821 non-null  object 
 3   activity_id          17821 non-null  int64  
 4   eprtrID              5404 non-null   object 
 5   parentCompany        6348 non-null   object 
 6   subsidiaryCompany    3725 non-null   object 
 7   permitID             16205 non-null  object 
 8   designatorICAO       618 non-null    object 
 9   monitoringID         1615 non-null   object 
 10  monitoringExpiry     0 non-null      float64
 11  monitoringFirstYear  1616 non-null   object 
 12  permitDateExpiry     4904 non-null   object 
 13  isAircraftOperator   17821 non-null  bool   
 14  ec748_2009Code       1615 non-null   object 
 15  permitDateEntry      16205 non-null 

Get account holder informations

In [4]:
df_accountHolder = get_account_holders(fn_zip)
df_accountHolder = df_accountHolder.rename(columns = {
    c: "accountHolder" + c.capitalize() for c in df_accountHolder.columns if c!= "id"})
df_accountHolder.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17981 entries, 0 to 17980
Data columns (total 8 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   id                                  17981 non-null  int64 
 1   accountHolderName                   17955 non-null  object
 2   accountHolderAddressmain            17923 non-null  object
 3   accountHolderPostalcode             17855 non-null  object
 4   accountHolderCity                   17940 non-null  object
 5   accountHolderLegalentityidentifier  1767 non-null   object
 6   accountHolderCountry_id             17902 non-null  object
 7   accountHolderCountry                17981 non-null  object
dtypes: int64(1), object(7)
memory usage: 1.1+ MB


Next, load the account data. The account dataframe includes the information of related look-up tables. If a dataframe with installation or account holder information is passed to the function, this information is also merged into the account table:

In [5]:
df_acc = get_accounts(fn_zip, 
                  df_installation=df_inst[["id", "name", "activity_id", "activity", "activityCategory", 
                                                   "nace_id", "nace", "naceCategory"]],
                    df_accountHolder=df_accountHolder)
df_acc.info()
# export the list with all accounts and related installation and account holder informations
df_acc.to_excel("eutl_account_full_list.xlsx", index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42445 entries, 0 to 42444
Data columns (total 29 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   id                                  42445 non-null  int64  
 1   name                                41618 non-null  object 
 2   registry_id                         42445 non-null  object 
 3   accountHolder_id                    41608 non-null  float64
 4   accountType_id                      41585 non-null  object 
 5   isOpen                              41612 non-null  object 
 6   openingDate                         41576 non-null  object 
 7   closingDate                         26283 non-null  object 
 8   commitmentPeriod                    40615 non-null  object 
 9   companyRegistrationNumber           37266 non-null  object 
 10  isRegisteredEutl                    41612 non-null  object 
 11  installation_id                     29201

Compliance data follow the same logic:

In [6]:
df_comp = get_compliance(fn_zip, df_installation=df_inst)
df_comp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 478031 entries, 0 to 478030
Data columns (total 46 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    478031 non-null  object 
 1   installation_id       478031 non-null  object 
 2   year                  478031 non-null  int64  
 3   reportedInSystem      478031 non-null  object 
 4   euetsPhase            478031 non-null  object 
 5   compliance_id         209014 non-null  object 
 6   allocatedFree         223739 non-null  float64
 7   allocatedNewEntrance  6886 non-null    float64
 8   allocatedTotal        478031 non-null  int64  
 9   allocated10c          1548 non-null    float64
 10  verified              190433 non-null  float64
 11  verifiedCummulative   198601 non-null  float64
 12  verifiedUpdated       478031 non-null  bool   
 13  surrendered           174288 non-null  float64
 14  complianceCode        209014 non-null  object 
 15  

Transactions follow the same logic also providing the descriptions provided in the look-up tables. In addition, we can pass a table with account information. The transaction loader also allows to resample the transaction data to a higher frequency: 

In [7]:
freq = "M"
# get transactions merged with account information
cols = ['id', 'name', 'registry_id', 'accountHolder_id', 'accountType_id',
        'isRegisteredEutl', 'installation_id', 'registry', 'accountType', 
        'accountCategory', 'installationName', 'activity_id', 'activity', 
        'nace_id', 'nace', "naceCategory" ]
df_trans = get_transactions(fn_zip, drop=["project_id"], freq=freq, 
                      df_account=df_acc[cols])
df_trans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1737918 entries, 0 to 1737917
Data columns (total 42 columns):
 #   Column                           Dtype         
---  ------                           -----         
 0   date                             datetime64[ns]
 1   id                               int64         
 2   transactionID                    object        
 3   transactionTypeMain_id           int64         
 4   transactionTypeSupplementary_id  int64         
 5   transferringAccount_id           float64       
 6   acquiringAccount_id              float64       
 7   unitType_id                      object        
 8   amount                           int64         
 9   transactionTypeMain              object        
 10  transactionTypeSupplementary     object        
 11  unitType                         object        
 12  transferringName                 object        
 13  transferringRegistry_id          object        
 14  transferringAccountHolder_id     f

We can wrap all of this into a single function, e.g.:

In [ ]:
def load_data(fn_zip, freq=None):
    """Load installation account and transaction data from zip file
    :param fn_zip: <string> name of zip file with eutl data
    :param freq: <string> frequency for resampling of transaction data"""
    # get installations
    df_inst = get_installations(fn_zip)
    # get accounts with selected installation information
    df_acc = get_accounts(fn_zip, 
                          df_installation=df_inst[["id", "name", "activity_id", "activity", "activityCategory", 
                                                   "nace_id", "nace", "naceCategory"]]) 
    # get compliance data with installation information
    df_comp = get_compliance(fn_zip, df_installation=df_inst)    
    # get transactions merged with account information
    cols = ['id', 'name', 'registry_id', 'accountHolder_id', 'accountType_id',
            'isRegisteredEutl', 'installation_id', 'registry', 'accountType', 
            'accountCategory', 'installationName', 'activity_id', 'activity', 
            'nace_id', 'nace', "naceCategory" ]
    df_trans = get_transactions(fn_zip, drop=["project_id"], freq=freq, 
                          df_account=df_acc[cols])
    return df_inst, df_acc, df_comp, df_trans

df_inst, df_acc, df_comp, df_trans = load_data(fn_zip, freq=None)